In [1]:
import sys
sys.path.append("..") # Ensure the parent directory is in the path

import core.assingment as Models
from core.DataLoader import DataPreprocessor, DataConfig
import core
import numpy as np
from importlib import reload
import keras
import matplotlib.pyplot as plt
MAX_JETS = 6
PLOTS_DIR = f"plots/"
import os
if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)

config = DataConfig(jet_features=["ordered_jet_pt","ordered_jet_eta", "ordered_jet_phi","ordered_jet_e","ordered_jet_b_tag"], 
                                lepton_features=["lep_pt", "lep_eta", "lep_phi", "lep_e"],
                                jet_truth_label="ordered_event_jet_truth_idx", 
                                lepton_truth_label="event_lepton_truth_idx", 
                                global_features = ["met_met_NOSYS","met_phi_NOSYS"], 
                                max_leptons=2, 
                                max_jets = MAX_JETS, 
                                non_training_features =["truth_ttbar_mass", "truth_ttbar_pt", "N_jets"], 
                                event_weight="weight_mc_NOSYS")

DataProcessor = DataPreprocessor(config)
DataProcessor.load_data("../../DATA/full_training.root", "reco", max_events=1000000)
DataProcessor.normalise_data()
X_train,y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1, random_state=42)

In [ ]:
reload(Models)
reload(core)
TransformerMatcher = Models.FeatureConcatTransformer(config, name="Transformer")

TransformerMatcher.build_model(
    num_heads=8,
    hidden_dim=64,
    num_layers=6,
    dropout_rate=0.1
)

TransformerMatcher.compile_model(
    loss = core.utils.AssignmentLoss(lambda_excl=0), optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4), metrics=[core.utils.AssignmentAccuracy()]
)
TransformerMatcher.model.summary()
#TransformerMatcher.load_model("Transformer_Assignment.keras")

Model: "FeatureConcatTransformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ global_inputs       │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ global_inputs[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ lep_inputs[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 6, 8)      │          0 │ flatten_1[0][0]   │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 15)     │          0 │ jet_inputs[0][0], │
│ (Concatenate)       │                   │            │ repeat_vector[0]… │
│                     │                   │            │ repeat_vector_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding (MLP) │ (None, 6, 64)     │      3,919 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6, 1)      │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 64)     │     33,472 │ jet_embedding[0]… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 64)     │     33,472 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 64)     │     33,472 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 64)     │     33,472 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 64)     │     33,472 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention_blo… │ (None, 6, 64)     │     33,472 │ self_attention_b… │
│ (SelfAttentionBloc… │                   │            │ jet_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_output_embeddi… │ (None, 6, 2)      │      1,440 │ self_attention_b… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 206,191 (805.43 KB)

 Trainable params: 206,191 (805.43 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
TransformerMatcher.train_model(epochs=50,
                                X_train=X_train,
                                y_train=y_train,
                                sample_weights=core.utils.compute_sample_weights(X_train, y_train),
                                batch_size=128,
                                callbacks = keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, restore_best_weights=True, mode ="min"))

Epoch 1/50
5609/5609 ━━━━━━━━━━━━━━━━━━━━ 183s 31ms/step - assignment_accuracy: 0.2641 - loss: 0.5600 - val_assignment_accuracy: 0.6163 - val_loss: 0.3860
Epoch 2/50
5609/5609 ━━━━━━━━━━━━━━━━━━━━ 181s 32ms/step - assignment_accuracy: 0.5749 - loss: 0.4122 - val_assignment_accuracy: 0.6617 - val_loss: 0.3552
Epoch 3/50
5609/5609 ━━━━━━━━━━━━━━━━━━━━ 187s 33ms/step - assignment_accuracy: 0.6175 - loss: 0.3872 - val_assignment_accuracy: 0.6649 - val_loss: 0.3430
Epoch 4/50
5609/5609 ━━━━━━━━━━━━━━━━━━━━ 193s 34ms/step - assignment_accuracy: 0.6335 - loss: 0.3733 - val_assignment_accuracy: 0.6753 - val_loss: 0.3339
Epoch 5/50
5609/5609 ━━━━━━━━━━━━━━━━━━━━ 200s 36ms/step - assignment_accuracy: 0.6452 - loss: 0.3663 - val_assignment_accuracy: 0.6910 - val_loss: 0.3269
Epoch 6/50
5609/5609 ━━━━━━━━━━━━━━━━━━━━ 202s 36ms/step - assignment_accuracy: 0.6549 - loss: 0.3574 - val_assignment_accuracy: 0.7030 - val_loss: 0.3138
Epoch 7/50
5609/5609 ━━━━━━━━━━━━━━━━━━━━ 204s 36ms/step - assignment_

In [ ]:
TransformerMatcher.save_model("Transformer_Assignment.keras")

In [ ]:
pred_val = TransformerMatcher.predict_indices(X_val)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
fig, ax = plt.subplots(figsize=(8,8))
ConfusionMatrixDisplay.from_predictions(y_val[:,:,1].argmax(axis=-1),pred_val[:,:,1].argmax(axis=-1), normalize="true", ax=ax)
plt.savefig(PLOTS_DIR+"/confusion_matrix_lepton.png")

In [ ]:
reload(core)
ml_eval = core.assingment.MLEvaluatorBase(TransformerMatcher, X_val, y_val)
importances = ml_eval.compute_permutation_importance(num_repeats=1)

In [ ]:
ml_eval.plot_feature_importance(importances)

In [ ]:
TransformerMatcher.export_to_onnx("Transformer_Assignment.onnx")

In [ ]:
import onnx
import onnxruntime as ort
onnx_model = onnx.load("Transformer_Assignment.onnx")
onnx.checker.check_model(onnx_model)
ort_session = ort.InferenceSession("Transformer_Assignment.onnx")
flatted_inputs = np.concatenate([X_val['jet'].reshape(X_val['jet'].shape[0], -1), X_val['lepton'].reshape(X_val['lepton'].shape[0], -1), X_val['global'].reshape(X_val["global"].shape[0],-1)], axis=-1).astype(np.float32)

In [ ]:
outputs = ort_session.run(None, {"flat_input": flatted_inputs})[0]

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_val[:,:,0].argmax(axis=-1),outputs[:,:,0].argmax(axis=-1), normalize="true")
